<a target="_blank" href="https://colab.research.google.com/github/vnstock-hq/vnstock_insider_guide/blob/main/demo/1_1_vnstock_data_explorer_v2_wrapper.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Initialization

In [11]:
from IPython.display import display
from vnstock import  Screener, Listing, Company, Finance,  Quote
from datetime import datetime, timedelta

In [12]:
screener_df = Screener().stock(params={"avgTradingValue20Day": (0.01, 10000)})

screener_df.head()

,ticker,exchange,industry,market_cap,roe,stock_rating,business_operation,business_model,financial_health,alpha,...,nim,price_vs_sma200,eps_ttm_growth1_year,eps_ttm_growth5_year,equity_mi,eps_recently,percent_price_vs_ma200,percent_price_vs_ma20,percent_price_vs_ma50,percent_price_vs_ma100
0,A32,UPCOM,Hàng cá nhân & Gia dụng,NaN,NaN,NaN,1.7,3.5,2.6,0.1,...,NaN,None,NaN,NaN,NaN,NaN,-0.2,-2.9,-4.3,-3.5
1,AAA,HSX,Hóa chất,3265.0,4.9,2.7,1.5,3.0,4.0,-0.1,...,NaN,None,-42.91,6.35,5249.0,692.0,9.8,6.2,13.3,16.7
2,AAH,UPCOM,Tài nguyên Cơ bản,483.0,2.1,2.5,1.1,NaN,4.4,0.0,...,NaN,"{'vi': 'Giá cắt xuống SMA(200)', 'en': 'Price ...",NaN,NaN,1205.0,209.0,0.8,-2.7,-0.1,-0.7
3,AAM,HSX,Thực phẩm và đồ uống,77.0,-0.1,NaN,1.1,NaN,4.2,0.0,...,NaN,None,NaN,NaN,199.0,-26.0,6.9,2.5,5.1,6.2
4,AAS,UPCOM,Dịch vụ tài chính,2878.0,2.6,3.3,3.8,2.8,3.2,0.1,...,NaN,None,-22.51,NaN,2541.0,285.0,48.8,16.6,28.3,39.4


In [13]:
listing = Listing()

listings_df = listing.symbols_by_industries()

listings_df = listings_df[listings_df['symbol'].isin(['BID', 'FPT', 'BIC', 'VND'])]
# Filter for symbol 
display(listings_df)

,symbol,organ_name,icb_name3,icb_name2,icb_name4,com_type_code,icb_code1,icb_code2,icb_code3,icb_code4
15,BID,Ngân hàng Thương mại Cổ phần Đầu tư và Phát tr...,Ngân hàng,Ngân hàng,Ngân hàng,NH,8301,8300,8350,8355
97,FPT,Công ty Cổ phần FPT,Phần mềm & Dịch vụ Máy tính,Công nghệ Thông tin,Phần mềm,CT,9000,9500,9530,9537
772,BIC,Tổng Công ty Cổ phần Bảo hiểm Ngân hàng Đầu tư...,Bảo hiểm phi nhân thọ,Bảo hiểm,Bảo hiểm phi nhân thọ,BH,8000,8500,8530,8536
1495,VND,Công ty Cổ phần Chứng khoán VNDIRECT,Dịch vụ tài chính,Dịch vụ tài chính,Môi giới chứng khoán,CK,8000,8700,8770,8777


In [18]:
from pathlib import Path
from time import sleep
current_folder = Path().cwd() / 'StockData'
today, eleven_years_ago = datetime.now(), datetime.now() - timedelta(days=365 * 11)
for symbol in listings_df["symbol"].tolist():
    # Get company profile, and financial statements
    print(f"Downloading {symbol}")
    symbol_folder = current_folder / symbol
    if not symbol_folder.exists():
        symbol_folder.mkdir()
    company = Company(symbol=symbol)
    company_profile_df = company.overview()
    company_profile_df["ticker"] = symbol
    # Delete symbol column from company profile
    company_profile_df = company_profile_df.drop("symbol", axis=1)
    company_financials = Finance(symbol=symbol, source="vci")
    balance_sheet_df = company_financials.balance_sheet(period="quarter", lang="en")
    income_statement_df = company_financials.income_statement(period="quarter", lang="en")
    cash_flow_df = company_financials.cash_flow(period="quarter", lang="en")
    ratio_df = Finance(symbol=symbol, source="tcbs").ratio(period="quarter", lang="en")

    company_profile_df.to_csv(symbol_folder / f"{symbol}_company_profile.csv")
    balance_sheet_df.to_csv(symbol_folder / f"{symbol}_balance_sheet.csv")
    income_statement_df.to_csv(symbol_folder / f"{symbol}_income_statement.csv")
    cash_flow_df.to_csv(symbol_folder / f"{symbol}_cash_flow.csv")
    ratio_df.to_csv(symbol_folder / f"{symbol}_ratio.csv")
    # Get trading data

    quote = Quote(symbol=symbol)
    quote_df = quote.history(start=eleven_years_ago.strftime("%Y-%m-%d"), end=today.strftime("%Y-%m-%d"), period="day")
    quote_df["ticker"] = symbol

    quote_df.to_csv(symbol_folder / f"{symbol}_quote.csv")

    sleep(5)

DataFrame.applymap has been deprecated. Use DataFrame.map instead.


DataFrame.applymap has been deprecated. Use DataFrame.map instead.


DataFrame.applymap has been deprecated. Use DataFrame.map instead.


DataFrame.applymap has been deprecated. Use DataFrame.map instead.
